In [142]:
import numpy as np
from scipy.sparse import coo_matrix, vstack, linalg
from scipy.sparse.linalg import cg
import mcubes
import open3d as o3d
import trimesh

In [143]:
def generate_point_cloud_with_normals(mesh, num_points=1000):
    # Randomly sample points from the mesh
    random_indices = np.random.choice(mesh.vertices.shape[0], num_points, replace=False)
    points = mesh.vertices[random_indices]
    normals = mesh.vertex_normals[random_indices]
    pc_mesh = trimesh.Trimesh(vertices=points, vertex_normals=normals)
    return pc_mesh

def save_point_cloud(mesh, mesh_file_name):
    # Save the point cloud as numpy array file, because Trimesh cannot preserve normals when saving point cloud 
    point_cloud_filename = "PointCloud_" + mesh_file_name.split('.')[0]
    np.savez(point_cloud_filename, vertices=mesh.vertices, vertex_normals=mesh.vertex_normals)

def load_point_cloud(mesh_file_name):
    # Load the point cloud from numpy array file
    input_point_cloud_filename = "PointCloud_" + mesh_file_name.split('.')[0] + ".npz"
    data = np.load(input_point_cloud_filename)
    return trimesh.Trimesh(vertices=data['vertices'], vertex_normals=data['vertex_normals'])

In [144]:
mesh_file_name = "bun_zipper_res3.ply"

if True:
    # Generate point cloud from mesh
    sample_num_points = 1000 # The number of points to sample from the mesh, that is the number of points in the point cloud.
    original_mesh = trimesh.load_mesh(mesh_file_name)
    point_cloud = generate_point_cloud_with_normals(original_mesh, num_points=sample_num_points)
    save_point_cloud(point_cloud, mesh_file_name)

In [145]:
input_point_cloud = load_point_cloud(mesh_file_name)
num_cubes = np.array([64, 64, 64])
padding = 4

input_points = input_point_cloud.vertices
input_normals = -input_point_cloud.vertex_normals # Reverse the normals because they are pointing inwards

def get_voxel_grid(vertices, num_cubes, padding):
    bounding_box = np.max(vertices, axis=0) - np.min(vertices, axis=0)
    print(bounding_box) # Bounding box means the maximum coordinates of the bounding box
    voxel_size = np.max(bounding_box) / num_cubes
    origin = np.min(vertices, axis=0) - padding * voxel_size
    voxel_num = num_cubes + 2 * padding
    return origin, voxel_size, voxel_num

origin, voxel_size, voxel_num = get_voxel_grid(input_points, num_cubes, padding)

print(origin) # Origin means the minimum coordinates of the bounding box
print(voxel_size) # Voxel size means the size of each voxel after dividing the bounding box into num_cubes
print(voxel_num) # Voxel resolution means the number of voxels in each dimension

[0.1535007  0.15139869 0.1187233 ]
[-0.10395809  0.02382051 -0.07062959]
[0.00239845 0.00239845 0.00239845]
[72 72 72]


In [146]:
def construct_staggered_gradient_matrix(n, axis, voxel_size):
    nx, ny, nz = n
    indices = np.arange(nx * ny * nz).reshape(n)
    if axis == 'x':
        idx_col = np.concatenate((indices[1:, :, :].flatten(), indices[:-1, :, :].flatten()))
        num_staggered = (nx - 1) * ny * nz
    elif axis == 'y':
        idx_col = np.concatenate((indices[:, 1:, :].flatten(), indices[:, :-1, :].flatten()))
        num_staggered = nx * (ny - 1) * nz
    else:  # axis == 'z'
        idx_col = np.concatenate((indices[:, :, 1:].flatten(), indices[:, :, :-1].flatten()))
        num_staggered = nx * ny * (nz - 1)

    row_idx = np.tile(np.arange(num_staggered), 2)
    data_terms = np.array([1 / voxel_size] * num_staggered + [-1 / voxel_size] * num_staggered)
    return coo_matrix((data_terms, (row_idx, idx_col)), shape=(num_staggered, nx * ny * nz))


def finite_difference_grad(num_voxel, size_voxel):
    hx, hy, hz = size_voxel
    Dx = construct_staggered_gradient_matrix(num_voxel, 'x', hx)
    Dy = construct_staggered_gradient_matrix(num_voxel, 'y', hy)
    Dz = construct_staggered_gradient_matrix(num_voxel, 'z', hz)

    G = vstack([Dx, Dy, Dz])
    return G

G = finite_difference_grad(voxel_num, voxel_size)

In [147]:
def trilinear_interpolation_weights(n_cube, corner, points, s_cube, direction=None):
    nx, ny, nz = n_cube
    hx, hy, hz = s_cube
    
    # grid coordinates / indices
    # minusing corner's coordinates makes the samples started from origin and the floor of them are the cubes' index
    # they are in. the auther call it as "grid coordinates/indices"

    x0 = np.floor((points[:, 0] - corner[0]) / hx).astype(int)  # (N, )
    y0 = np.floor((points[:, 1] - corner[1]) / hy).astype(int)  # (N, )
    z0 = np.floor((points[:, 2] - corner[2]) / hz).astype(int)  # (N, )
    x1 = x0 + 1  # (N, )
    y1 = y0 + 1  # (N, )
    z1 = z0 + 1  # (N, )

    # the coordinates of samples in their local cubes, the numeral values of them are the percentages along each axis.
    xd = (points[:, 0] - corner[0]) / hx - x0  # (N, )
    yd = (points[:, 1] - corner[1]) / hy - y0  # (N, )
    zd = (points[:, 2] - corner[2]) / hz - z0  # (N, )

    # data terms for the trilinear interpolation weight matrix
    weight_000 = (1 - xd) * (1 - yd) * (1 - zd)
    weight_100 = xd * (1 - yd) * (1 - zd)
    weight_010 = (1 - xd) * yd * (1 - zd)
    weight_110 = xd * yd * (1 - zd)
    weight_001 = (1 - xd) * (1 - yd) * zd
    weight_101 = xd * (1 - yd) * zd
    weight_011 = (1 - xd) * yd * zd
    weight_111 = xd * yd * zd
    data_term = np.concatenate((weight_000,
                                weight_100,
                                weight_010,
                                weight_110,
                                weight_001,
                                weight_101,
                                weight_011,
                                weight_111))

    row_idx = np.arange(points.shape[0])
    row_idx = np.tile(row_idx, 8)

    if direction == "x":
        num_grids = (nx - 1) * ny * nz
        staggered_grid_idx = np.arange((nx - 1) * ny * nz).reshape((nx - 1, ny, nz))
    elif direction == "y":
        num_grids = nx * (ny - 1) * nz
        staggered_grid_idx = np.arange(nx * (ny - 1) * nz).reshape((nx, ny - 1, nz))
    elif direction == "z":
        num_grids = nx * ny * (nz - 1)
        staggered_grid_idx = np.arange(nx * ny * (nz - 1)).reshape((nx, ny, nz - 1))
    else:
        num_grids = nx * ny * nz
        staggered_grid_idx = np.arange(nx * ny * nz).reshape((nx, ny, nz))

    col_idx_000 = staggered_grid_idx[x0, y0, z0]
    col_idx_100 = staggered_grid_idx[x1, y0, z0]
    col_idx_010 = staggered_grid_idx[x0, y1, z0]
    col_idx_110 = staggered_grid_idx[x1, y1, z0]
    col_idx_001 = staggered_grid_idx[x0, y0, z1]
    col_idx_101 = staggered_grid_idx[x1, y0, z1]
    col_idx_011 = staggered_grid_idx[x0, y1, z1]
    col_idx_111 = staggered_grid_idx[x1, y1, z1]
    col_idx = np.concatenate((col_idx_000,
                                col_idx_100,
                                col_idx_010,
                                col_idx_110,
                                col_idx_001,
                                col_idx_101,
                                col_idx_011,
                                col_idx_111))

    W = coo_matrix((data_term, (row_idx, col_idx)), shape=(points.shape[0], num_grids))
    return W

print(voxel_num, origin, input_points, voxel_size)
Wx = trilinear_interpolation_weights(voxel_num, origin, input_points, voxel_size, direction="x")
Wy = trilinear_interpolation_weights(voxel_num, origin, input_points, voxel_size, direction="y")
Wz = trilinear_interpolation_weights(voxel_num, origin, input_points, voxel_size, direction="z")
W = trilinear_interpolation_weights(voxel_num, origin, input_points, voxel_size)

[72 72 72] [-0.10395809  0.02382051 -0.07062959] [[-0.0861137   0.0814707   0.00908143]
 [-0.0690323   0.0427133   0.00739115]
 [-0.0274495   0.179802   -0.00925837]
 ...
 [-0.00876679  0.128727    0.025102  ]
 [-0.0515867   0.13164    -0.0028092 ]
 [ 0.0562736   0.0586365   0.0253398 ]] [0.00239845 0.00239845 0.00239845]


In [148]:
V = np.concatenate(
    [
        Wx.T @ input_normals[:, 0],
        Wy.T @ input_normals[:, 1],
        Wz.T @ input_normals[:, 2],
    ]
)

In [149]:
g, _ = cg(G.T @ G, G.T @ V, maxiter=2000, tol=1e-5)
g -= np.mean(W @ g)
g_field = g.reshape(voxel_num)
vertices, triangles = mcubes.marching_cubes(g_field, 0)

In [150]:
vertices[:, 0] = vertices[:, 0] * voxel_size[0] + origin[0]
vertices[:, 1] = vertices[:, 1] * voxel_size[1] + origin[1]
vertices[:, 2] = vertices[:, 2] * voxel_size[2] + origin[2]

output_mesh = trimesh.Trimesh(vertices=vertices, faces=triangles)
output_mesh.export("output_mesh.ply")

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 14114\nproperty float x\nproperty float y\nproperty float z\nelement face 28224\nproperty list uchar int vertex_indices\nend_header\n\xdd:\xbf\xbdpZ\xf0=\xc8+Z<\x98X\xbc\xbd\xff\x9f\xee=\xc8+Z<\x98X\xbc\xbdpZ\xf0=\x1c\x993<\x94x\xbf\xbdpZ\xf0=\xcd\xbb\x80<\x98X\xbc\xbd\x13\xf6\xee=\xcd\xbb\x80<\xbd\xfc\xbc\xbdpZ\xf0=\xb6a\x94<\x98X\xbc\xbd\x7f\x90\xef=\xb6a\x94<\xe9\xf7\xbc\xbdpZ\xf0=\x9f\x07\xa8<\x98X\xbc\xbd\xe6\xfd\xed=\x9f\x07\xa8<\x98X\xbc\xbdpZ\xf0=y^\xb0<\x97\xb8\xbc\xbd\xeaC\xf5=\xf5\xdf2<\x98X\xbc\xbd\x10j\xf1=\xf5\xdf2<\x98X\xbc\xbd\xeaC\xf5=\x17a1<\x02\x05\xbf\xbd\xeaC\xf5=\xc8+Z<R\xfa\xbe\xbd\xeaC\xf5=\xcd\xbb\x80<n\x1b\xbe\xbd\xeaC\xf5=\xb6a\x94<\xe2\x0f\xbe\xbd\xeaC\xf5=\x9f\x07\xa8<\xde}\xbc\xbd\xeaC\xf5=\x89\xad\xbb<\x98X\xbc\xbd\x1b\xe5\xf4=\x89\xad\xbb<\x98X\xbc\xbd\xeaC\xf5=\x03\xec\xbc<\xff\\\xbd\xbde-\xfa=\xf5\xdf2<\x98X\xbc\xbde-\xfa=\x94m!<\x84b\xbe\xbde-\xfa=\xc8+Z<

In [151]:
def build_open3d_octree(points, depth):
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    
    octree = o3d.geometry.Octree(max_depth=depth)
    octree.convert_from_point_cloud(point_cloud, size_expand=0.01)
    return octree


depth = 8  # Depth of the octree
points = input_points
normals = input_normals

octree = build_open3d_octree(points, depth)
octree.traverse(lambda node, _node_info: print(node))

OctreeInternalPointNode with 8 non-empty child nodes and 1000 points
OctreeInternalPointNode with 4 non-empty child nodes and 125 points
OctreeInternalPointNode with 2 non-empty child nodes and 12 points
OctreeInternalPointNode with 3 non-empty child nodes and 4 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreePointColorLeafNode with color [0, 0, 0] containing 1 points.
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreePointColorLeafNode with color [0, 0, 0] containing 1 points.
OctreeInternalPointNode with 2 non-empty child nodes and 2 points
Oc

In [152]:
def use_octree_for_reconstruction(octree, points, normals):
    pass

## Comparison of Open3D methods for point cloud reconstruction

In [164]:
# https://www.open3d.org/docs/release/tutorial/geometry/surface_reconstruction.html

def test_alpha_shapes(mesh_name, num_sample_points=1000):
    mesh = o3d.io.read_triangle_mesh(mesh_name)
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_poisson_disk(num_sample_points)

    alpha = 0.03 # the tradeoff parameter 
    print(f"alpha={alpha:.3f}")
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)




def test_ball_pivoting(mesh_name, num_sample_points=1000):
    mesh = o3d.io.read_triangle_mesh(mesh_name)
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_poisson_disk(num_sample_points)

    radii = [0.005, 0.01, 0.02, 0.04] # parameter that corresponds to the radii of the individual balls that are pivoted on the point cloud.
    print(f"radii={radii}")
    rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd, o3d.utility.DoubleVector(radii))
    o3d.io.write_triangle_mesh("test_ball_pivoting-" + mesh_name, rec_mesh)
    o3d.visualization.draw_geometries([pcd, rec_mesh])


test_alpha_shapes("bun_zipper_res3.ply")
test_ball_pivoting("bun_zipper_res3.ply")

alpha=0.030
radii=[0.005, 0.01, 0.02, 0.04]
